In [3]:
from models.nac import NAC
import torch.nn.functional as F
import torch
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from tqdm import tqdm_notebook as tqdm

from data_generator_helper import generate_synthetic_selection_dataset_with_function

from models.nac import NAC
from models.nalu import NALU
from models.mlp import MLP
import torch
import torchvision.utils as vutils
import numpy as np
import torchvision.models as models
from torchvision import datasets
from tensorboardX import SummaryWriter
import datetime
import os

In [4]:
def reportLoss(loss, epoch):
    pass
    #print("epoch {},  \t loss {}".format(epoch, loss))
    
def train(model, optimizer, x_train, y_train, epochs, batch_size):
    
    for epoch in range(epochs):
    
        
        for batch in range(len(x_train) // batch_size):
            
            model.train()
            optimizer.zero_grad()

            x_batch_train = x_train[batch:(batch+batch_size),:]
            y_batch_train = y_train[batch:(batch+batch_size)].unsqueeze(1)

            out = model(x_batch_train)

            loss = F.mse_loss(out, y_batch_train)
            
            if loss != loss:
                break
                print("nan detected")
            
            loss.backward()
            optimizer.step()
        
        if loss != loss:
            break
        
        if epoch % 10 == 0: 
            #pass
            reportLoss(loss.data, epoch)
            
    return test(model,x_train,y_train)
        
def test(model, x_test, y_test):
    
    model.eval()
    output_test = model(x_test)
    #print(np.shape(output_test), np.shape(y_test))
    loss = F.mse_loss(output_test.squeeze(), y_test)
    
    return loss

## Learning rate

In [5]:
test_per_range = 5
sample_size = 100
set_size = 100

in_dim = sample_size
hidden_dim = 2
out_dim = 1
num_layers = 2

lrs = [0.1, 0.01, 0.001, 0.0001]
epochs = 1000
batch_size = 1

#values = np.linspace(1,1000,10)
min_value = 0
max_value = 1


functions = ['add', 'substraction','multiplication','division']

train_acc = 0
test_acc = 0

train_loss = np.zeros((len(functions),len(lrs),test_per_range))
test_loss = np.zeros((len(functions),len(lrs),test_per_range))

for j, function in enumerate(functions):
    print(function)
    for k, lr in enumerate(lrs):
        for i in range(test_per_range):

            model = NALU(2,in_dim,2,out_dim)
            optimizer = torch.optim.RMSprop(model.parameters(),lr=lr)

            x_train, y_train, boundaries = generate_synthetic_selection_dataset_with_function(min_value, max_value,
                                                                        sample_size, set_size, function, boundaries = None)

            x_test, y_test, _ = generate_synthetic_selection_dataset_with_function(min_value, 100,
                                                                        sample_size, set_size, function, boundaries = boundaries)

            train_loss[j,k, i] = train(model, optimizer, x_train, y_train, epochs, batch_size)

            test_loss[j, k, i]  = test(model, x_test, y_test)

print(train_loss, test_loss)

np.save('learning_rate_interpolation_train_nalu.npy', train_loss)
np.save('learning_rate_extrapolation_test_nalu.npy', test_loss)



add
substraction
multiplication
division
[[[2.01544025e-11 2.32466841e-11 6.86933547e-02            nan
   3.68890686e+01]
  [6.00231942e-05 4.17416324e-08 6.51249737e-02 2.01839242e-07
   3.12891646e-09]
  [           nan 2.73191482e-02 1.15073711e-01 4.85689193e-01
   5.61145786e-03]
  [1.21695288e-04 1.09930756e-02 1.08744670e-02 1.73223972e+00
   4.68190063e+02]]

 [[4.54125595e+00 1.98383532e-11 5.93203926e+00 2.31375457e-11
   2.02442160e-11]
  [7.35967660e-08 4.87756282e-02 3.78143676e-02 1.17270443e-04
   4.10744087e-06]
  [5.29329330e-02 2.74891436e-01 1.29371102e-03 2.98933894e-03
   6.92122383e-03]
  [2.08523840e-01 5.65112472e-01 8.53688344e-02 1.79406966e-03
   1.38660129e-02]]

 [[           nan 8.37080383e+00 1.22034473e+03 7.86734543e+01
              nan]
  [8.81792605e-02 2.65240250e+01            nan 4.13525677e+00
   1.23499455e-02]
  [2.42251389e+02 6.95940316e-01 7.19340706e+00 2.70519066e+00
   1.60902354e-03]
  [1.11257541e+00 2.48295502e+02            nan 7.499

# Batch size

In [7]:
test_per_range = 5
sample_size = 100
set_size = 100

in_dim = sample_size
hidden_dim = 2
out_dim = 1
num_layers = 2

lrs = 0.01
epochs = 1000
batch_sizes = [1, 2, 4, 8]

#values = np.linspace(1,1000,10)
min_value = 0
max_value = 1


functions = ['add', 'substraction','multiplication','division']

train_acc = 0
test_acc = 0

train_loss = np.zeros((len(functions),len(batch_sizes),test_per_range))
test_loss = np.zeros((len(functions),len(batch_sizes),test_per_range))

for j, function in enumerate(functions):
    print(function)
    for k, batch_size in enumerate(batch_sizes):
        for i in range(test_per_range):

            model = NALU(2,in_dim,2,out_dim)
            optimizer = torch.optim.RMSprop(model.parameters(),lr=lr)

            x_train, y_train, boundaries = generate_synthetic_selection_dataset_with_function(min_value, max_value,
                                                                        sample_size, set_size, function, boundaries = None)

            x_test, y_test, _ = generate_synthetic_selection_dataset_with_function(min_value, 100,
                                                                        sample_size, set_size, function, boundaries = boundaries)

            train_loss[j,k, i] = train(model, optimizer, x_train, y_train, epochs, batch_size)

            test_loss[j, k, i]  = test(model, x_test, y_test)

print(train_loss, test_loss)

np.save('batch_sizes_interpolation_train_nalu.npy', train_loss)
np.save('batch_sizes_extrapolation_test_nalu.npy', test_loss)



add
substraction
multiplication
division
[[[2.28459299e-01 1.42108455e-01 2.28919964e-02 1.14229284e-02
   1.37843576e-03]
  [1.00605001e+01 3.23533020e+01 7.48428879e+01 3.49333191e+02
   4.17449331e+00]
  [6.92352829e+01 1.09259692e+03 1.54937164e+02 5.13519165e+02
   7.82524048e+02]
  [2.95464404e+03 1.67556885e+03 1.50211758e+04 1.62367904e+00
   1.78779251e+02]]

 [[2.93904257e+00 6.05433047e-01 1.21979609e-01 3.86272930e-03
   4.88726974e-01]
  [5.54678392e+00 6.31649196e-01 1.97292268e+00 4.88766602e+02
   1.81287022e+01]
  [1.23168594e+02 1.56951857e+01 3.19370880e+02 2.69258747e+01
   9.40450609e-01]
  [2.70213320e+07 1.43917615e+03 1.40442780e+02 9.80852661e+01
   9.20733452e+00]]

 [[7.02821045e+02 4.73959991e+02 7.53028440e+00 1.46551390e+01
   6.29543513e-02]
  [1.08277855e+01 1.39102188e+04 5.15361670e+03 1.86355298e+03
   4.94807383e+04]
  [1.24981572e+04 5.68082500e+04 9.67203125e+03 5.45255562e+05
   3.29660742e+03]
  [2.16007007e+03 7.69408203e+03 6.68084521e+03 9.457

# Sample size

In [11]:
test_per_range = 5
sample_size = 100
set_sizes = [100,1000,10000]

in_dim = sample_size
hidden_dim = 2
out_dim = 1
num_layers = 2

lrs = 0.01
epochs = 5000
batch_sizes = 1

#values = np.linspace(1,1000,10)
min_value = 0
max_value = 1

functions = ['add', 'substraction','multiplication','division']

train_acc = 0
test_acc = 0

train_loss = np.zeros((len(functions),len(sample_sizes),test_per_range))
test_loss = np.zeros((len(functions),len(sample_sizes),test_per_range))

for j, function in enumerate(functions):
    print(function)
    for k, set_size in enumerate(set_sizes):
        print(set_size)
        for i in range(test_per_range):

            model = NALU(2,in_dim,2,out_dim)
            optimizer = torch.optim.RMSprop(model.parameters(),lr=lr)

            x_train, y_train, boundaries = generate_synthetic_selection_dataset_with_function(min_value, max_value,
                                                                        sample_size, set_size, function, boundaries = None)

            x_test, y_test, _ = generate_synthetic_selection_dataset_with_function(min_value, 100,
                                                                        sample_size, set_size, function, boundaries = boundaries)

            train_loss[j,k, i] = train(model, optimizer, x_train, y_train, epochs, batch_size)

            test_loss[j, k, i]  = test(model, x_test, y_test)

print(train_loss, test_loss)

np.save('sample_interpolation_train_nalu.npy', train_loss)
np.save('sample_extrapolation_test_nalu.npy', test_loss)

add
100
1000
10000
substraction
100
1000
10000
multiplication
100
1000
10000
division
100
1000
10000
[[[6.14437793e+03 8.02575012e+02 5.29618896e+02 1.58751826e+04
   1.02213602e+01]
  [1.81133650e+06 4.19657249e+01 2.50613330e+03 1.84560928e+01
   1.03325391e+03]
  [           inf 4.29219824e+03 3.94171834e-01            nan
   1.10560050e-03]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
   0.00000000e+00]]

 [[3.34871758e+04 3.46560120e+02 4.48694427e+02 3.49600372e+01
   5.90602827e+00]
  [7.70838594e+04 1.10592727e+05 2.92097549e+01 3.49230408e+02
   1.00827265e+00]
  [7.27312040e+00 2.06671844e+02 1.36207032e+00 1.84816837e+01
   8.68103532e+17]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
   0.00000000e+00]]

 [[5.54636803e+01 1.11672583e+03 2.70993404e+01 1.42069938e+05
   1.57296112e+06]
  [7.10740234e+03 5.55934688e+04 3.52995117e+04 1.42359680e+02
   3.28295840e+07]
  [7.35377991e+02            inf 1.93841895e+04            nan
   5.3913720